In [ ]:
import subprocess
import os
from datetime import datetime

def export_msinfo_report(output_path=None):
    if output_path is None:
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        output_path = os.path.join(os.getcwd(), f"msinfo32_report_{timestamp}.xml")

    command = ["msinfo32.exe", "/nfo", output_path]

    try:
        subprocess.run(command, check=True)
        print(f"Отчёт успешно сохранён: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при запуске msinfo32: {e}")

export_msinfo_report()


Отчёт успешно сохранён: d:\miwn36\Python\Coursework\CourseWork-backend\msinfo32_report_2025-10-30_13-53-50.xml


In [6]:
import xml.etree.ElementTree as ET
import csv
import os
from collections import defaultdict

def flatten_element(element, parent_path=""):
    """Разворачивает XML-элемент в словарь {путь: значение}"""
    data = {}
    path = f"{parent_path}/{element.tag}" if parent_path else element.tag

    # Текст
    text = (element.text or "").strip()
    if text:
        data[path] = text

    # Атрибуты
    for attr, val in element.attrib.items():
        data[f"{path}@{attr}"] = val

    # Дети
    for child in element:
        child_data = flatten_element(child, path)
        data.update(child_data)

    return data


def xml_to_csv(xml_file, output_dir="output_csv"):
    os.makedirs(output_dir, exist_ok=True)

    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Словарь: {имя_тега: список_строк}
    categories = defaultdict(list)

    # Рекурсивный поиск повторяющихся элементов
    def collect_records(element):
        # Если у элемента есть несколько одноимённых детей — это "категория"
        child_tags = [child.tag for child in element]
        for tag in set(child_tags):
            same = [c for c in element if c.tag == tag]
            if len(same) > 1:  # повторяющийся тег = записи
                for item in same:
                    categories[tag].append(flatten_element(item))
        # Рекурсивно вниз
        for child in element:
            collect_records(child)

    collect_records(root)

    # Записываем каждую категорию в CSV
    for cat, rows in categories.items():
        fieldnames = sorted({k for row in rows for k in row.keys()})
        csv_file = os.path.join(output_dir, f"{cat}.csv")
        with open(csv_file, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
        print(f"Категория '{cat}' → {len(rows)} строк в {csv_file}")


if __name__ == "__main__":
    xml_to_csv("msinfo32_report_2025-10-30_13-53-50.xml")


Категория 'Category' → 46 строк в output_csv\Category.csv
Категория 'Data' → 2748 строк в output_csv\Data.csv
Категория 'Состояние' → 790 строк в output_csv\Состояние.csv
